In [ ]:
import os
import cv2
import subprocess
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import FileLink, display

INPUT_DIR = "/kaggle/input/tattoos"
image_paths = os.listdir(INPUT_DIR)
image_paths = [os.path.join(INPUT_DIR, path) for path in image_paths]

In [ ]:
def download_file(folder="/kaggle/working"):
    command = f"zip out.zip {folder} -r"
    subprocess.run(command, shell=True)


def clear_dir(folder="/kaggle/working"):
    for the_file in os.listdir(folder):
        file_path = os.path.join(folder, the_file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                clear_dir(file_path)
                os.rmdir(file_path)
        except Exception as e:
            print(e)


def visualize(*imgs):
    for i in range(len(imgs)):
        plt.subplot(2, 2, i + 1)
        plt.imshow(imgs[i], 'gray')


def get_contour_mask(image, threshed):
    contours, H = cv2.findContours(image=threshed, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=lambda x: cv2.contourArea(x), reverse=True)
    rects = [cv2.boundingRect(contour) for contour in contours[:5]]
    mask = np.zeros(image.shape[:2], np.uint8)
    for x, y, w, h in rects:
        mask[x:x+w, y:y+h] = 255
    return mask


def remove_background(image_paths):
    for path in image_paths:
        src = cv2.imread(path, 1)
        tmp = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

        mean = round(cv2.mean(tmp)[0])
        algo = cv2.THRESH_BINARY if mean < 127 else cv2.THRESH_BINARY_INV
        _, thr = cv2.threshold(tmp, mean, 255, algo)
        
        mask = get_contour_mask(src, thr)
        cv2.bitwise_and(thr, thr, mask=mask)
        final = cv2.merge([*cv2.split(src), thr], 4)

        cv2.imwrite(f"output{len(os.listdir())}.png", final)


# clear_dir()
# download_file()
# remove_background(image_paths)